ISO-8859-1


In [ ]:
import pandas as pd
import csv
# I want to figure out the row count of a csv file
df = pd.read_csv(file, encoding='ISO-8859-1', quoting=csv.QUOTE_NONE)
len(df)


In [1]:
test = 3 * 8
print(test)
# This is a test cell to be deleted

24
